In [1]:
import numpy as np
from music21 import *
import glob
import sys
import math
from tensorflow.keras.models import Sequential, Model, load_model
sys.path.append("../RL modules/")
from env_noOctave import SegmentationEnv

In [2]:
base_lr = 0.001
max_lr = 0.1
step_size = 8
def masterScheduler(epoch):
    def scheduler():
            period = 2 * step_size
            cycle = math.floor(1 + epoch/period)
            x = abs(epoch/step_size - 2*cycle + 1)
            delta = (max_lr - base_lr)*max(0, (1 - x))
            delta /= float(2 ** (cycle - 1))
            return base_lr + delta
    return scheduler

In [3]:
for i in range(20):
    print(masterScheduler(i)())

0.001
0.013375000000000001
0.025750000000000002
0.038125000000000006
0.0505
0.062875
0.07525000000000001
0.08762500000000001
0.1
0.08762500000000001
0.07525000000000001
0.062875
0.0505
0.038125000000000006
0.025750000000000002
0.013375000000000001
0.001
0.0071875
0.013375000000000001
0.019562500000000003


In [16]:
testing_pieces = []
# for piece in glob.glob('./normal/training/*'):
#     testing_pieces.append(piece)

for piece in glob.glob('./C_normalized/testing/*'):
    testing_pieces.append(piece)

In [17]:
model = load_model("dqn_normalized")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 48)                1248      
_________________________________________________________________
dense_1 (Dense)              (None, 24)                1176      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 2,474
Trainable params: 2,474
Non-trainable params: 0
_________________________________________________________________


In [32]:
env = SegmentationEnv(testing_pieces)

./C_normalized/testing\trans_Il_Vecchio_Castello.musicxml
./C_normalized/testing\trans_Minuet_in_G_Major.musicxml
./C_normalized/testing\trans_Prélude_in_B_Minor.musicxml
./C_normalized/testing\trans_Sonate_No._31.musicxml
./C_normalized/testing\trans_Étude_in_Gb_Major_Op25.musicxml
Total number of pieces 5


In [19]:
len(env.notes),len(testing_pieces)

(5, 5)

In [35]:
def Visualization(piece,offset): #offset assume sorted.
    def cal_offset(e):
        if e is None:
            return 0
        return e.offset + cal_offset(e.activeSite)
    c = converter.parse(piece)
    all_notes = []
    for el in c.recurse().notes:
        if el.lyric is not None:
            el.lyric = el.lyric.replace("♭", "b")
        all_notes.append([el.lyric, el, cal_offset(el)])
    b = sorted(all_notes, key=lambda x: x[0] if x[0] is not None else "ZZZ")
    b = sorted(b, key=lambda x: x[-1])
    for e in b:
        if len(offset) >0 and float(round(e[2],1)) >= offset[0]:
            del offset[0]
            e[1].addLyric("CUT")
    GEX = musicxml.m21ToXml.GeneralObjectExporter(c)
    out = GEX.parse()
    piecename = piece.split("\\")[-1]
    piecename = piecename.split(".")[0]
    piecename = "Vis_"+piecename + ".musicxml"
    print(piece,piecename)
    with open(piecename, "wb") as f:
        f.write(out)

In [37]:
for i in range(len(env.notes)):
    offsets = []
    obs = env.reset(i)
    total_reward = 0
    max_reward = 0
    num_correct_segment = 0
    while True:
        obs = obs.reshape((1, 12*2+1))
        action = np.argmax(model.predict(obs))
        offsets.append(env.current_noteoffset)
        obs, reward, done, info = env.step(action)
        #env.render()
        total_reward += reward
        max_reward += 1
        if reward ==1 and action == 1:
            num_correct_segment += 1
        if done:
            break
    print(f"Piece {testing_pieces[i]}, total reward = {total_reward}/{max_reward}, correct segment = {num_correct_segment}")
    print(offset)
    offsets = [float(round(offset,1)) for offset in offsets]
    print(offsets)
#     Visualization(env.piecelist[i],offsets)

Piece ./C_normalized/testing\trans_Il_Vecchio_Castello.musicxml, total reward = 61/205, correct segment = 77


NameError: name 'offset' is not defined